In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

# Configuration

In [61]:
MODEL_DIR = 'model'
DATA_FILE_NAME = 'new_traffic_data.csv'
CREATE_MODEL = True

# Import data from csv

In [3]:
df = pd.read_csv(DATA_FILE_NAME)
df['time'] = df['time'].astype('datetime64[ns]')
df.dtypes

time                  datetime64[ns]
currentSpeed                   int64
weatherDesc                   object
motorcycles                    int64
cars                           int64
heavyVehicles                  int64
half_hour_interval            object
dtype: object

# Add new columns for further analysis

In [4]:
df['total_vehicle'] = df['cars'] + df['motorcycles'] + df['heavyVehicles']
df['Day'] = df['time'].dt.day_name()
df

time  currentSpeed       weatherDesc  motorcycles  cars  \
0   2025-03-12 10:30:03            46  scattered clouds            0    13   
1   2025-03-12 11:00:04            49  scattered clouds            1     9   
2   2025-03-12 11:30:04            45  scattered clouds            1    13   
3   2025-03-12 12:00:04            49  scattered clouds            0     5   
4   2025-03-12 12:30:04            51  scattered clouds            4    11   
..                  ...           ...               ...          ...   ...   
928 2025-04-01 22:01:40            66        few clouds            0     0   
929 2025-04-01 22:31:40            66        few clouds            0     1   
930 2025-04-01 23:01:40            66        few clouds            0     0   
931 2025-04-01 23:31:40            66        light rain            0     3   
932 2025-04-02 00:01:40            66        few clouds            0     0   

     heavyVehicles half_hour_interval  total_vehicle        Day  
0                0              10:30             13  Wednesday  
1                0              11:00             10  Wednesday  
2                0              11:30             14  Wednesday  
3                0              12:00              5  Wednesday  
4                0              12:30             15  Wednesday  
..             ...                ...            ...        ...  
928              0              22:00              0    Tuesday  
929              0              22:30              1    Tuesday  
930              0              23:00              0    Tuesday  
931              0              23:30              3    Tuesday  
932              0              00:00              0  Wednesday  

[933 rows x 9 columns]

# EDA

In [5]:
df.describe()

time  currentSpeed  motorcycles        cars  \
count                            933    933.000000   933.000000  933.000000   
mean   2025-03-22 19:47:56.619506944     54.997856     0.875670    7.079314   
min              2025-03-12 10:30:03     22.000000     0.000000    0.000000   
25%              2025-03-17 10:00:26     46.000000     0.000000    0.000000   
50%              2025-03-23 05:00:14     56.000000     0.000000    6.000000   
75%              2025-03-28 03:00:57     66.000000     1.000000   11.000000   
max              2025-04-02 00:01:40     66.000000    13.000000   39.000000   
std                              NaN     12.143897     1.652017    7.771974   

       heavyVehicles  total_vehicle  
count     933.000000     933.000000  
mean        0.248660       8.203644  
min         0.000000       0.000000  
25%         0.000000       0.000000  
50%         0.000000       6.000000  
75%         0.000000      14.000000  
max         3.000000      46.000000  
std         0.595324       8.962446

In [6]:
VEHICLE_FEATURE = ['cars', 'motorcycles',  'heavyVehicles']
FLOW_FEATURE = ['currentSpeed'] 

## Line chart showing relation between number of each type of vehicle and time

In [7]:
# # Additional 30 min period column
# base_time = pd.to_datetime('00:00:00')
# total_minute = ((df['time'].dt.minute + (df['time'].dt.hour * 60)) // 30) * 30 
# df['half_hour_interval'] = (base_time + pd.to_timedelta(total_minute, unit='m')).dt.time

In [8]:
px.line(df, x="time", y=VEHICLE_FEATURE)


## Line chart showing relation between number of each type of vehicle and time

In [9]:
px.line(df, x="time", y=FLOW_FEATURE)


## FLow and number of each vehicle on a specific date

In [10]:
def relation_with_time(selected_attr, title='title', xla='x', yla='y'):
    filtered_df = df.loc[df['time'].dt.date == pd.Timestamp('2025-03-30').date()]
    fig = px.line(filtered_df, x="time", y=selected_attr)
    fig.update_layout(
        title=title,
        xaxis_title=xla,
        yaxis_title=yla
    )
    fig.show()
    

relation_with_time(VEHICLE_FEATURE, 'Vehicle count on specific day', 'Time', 'Vehicle count')
relation_with_time(FLOW_FEATURE, 'Traffic flow on specific day', 'Time', 'Traffic flow')

df

time  currentSpeed       weatherDesc  motorcycles  cars  \
0   2025-03-12 10:30:03            46  scattered clouds            0    13   
1   2025-03-12 11:00:04            49  scattered clouds            1     9   
2   2025-03-12 11:30:04            45  scattered clouds            1    13   
3   2025-03-12 12:00:04            49  scattered clouds            0     5   
4   2025-03-12 12:30:04            51  scattered clouds            4    11   
..                  ...           ...               ...          ...   ...   
928 2025-04-01 22:01:40            66        few clouds            0     0   
929 2025-04-01 22:31:40            66        few clouds            0     1   
930 2025-04-01 23:01:40            66        few clouds            0     0   
931 2025-04-01 23:31:40            66        light rain            0     3   
932 2025-04-02 00:01:40            66        few clouds            0     0   

     heavyVehicles half_hour_interval  total_vehicle        Day  
0                0              10:30             13  Wednesday  
1                0              11:00             10  Wednesday  
2                0              11:30             14  Wednesday  
3                0              12:00              5  Wednesday  
4                0              12:30             15  Wednesday  
..             ...                ...            ...        ...  
928              0              22:00              0    Tuesday  
929              0              22:30              1    Tuesday  
930              0              23:00              0    Tuesday  
931              0              23:30              3    Tuesday  
932              0              00:00              0  Wednesday  

[933 rows x 9 columns]

## Average number of each type of vehicle and traffic flow in each day of week

In [1]:
def avg_on_day_of_week(selected_attr, title='title', xla='x', yla='y'):
    
    color_map = {
        "Monday": "yellow",
        "Tuesday": "pink",
        "Wednesday": "green",
        "Thursday": "orange",
        "Friday": "blue",
        "Saturday": "purple",
        "Sunday": "red"
    }

    # Extract day names
    df['Day'] = df['time'].dt.day_name()

    grouped = df.groupby(['Day', 'half_hour_interval'])[selected_attr].mean().reset_index().sort_values('half_hour_interval', ascending=True)
    
    fig = px.line(
        grouped, 
        y=selected_attr, 
        x='half_hour_interval',
        color='Day',
        color_discrete_map=color_map,
    )
    fig.update_layout(
        xaxis=dict(title=xla),
        title=title,
        yaxis_title=yla
    )
    fig.show()
    
avg_on_day_of_week(['total_vehicle'], 'Average vehicle count on each day of week every 30 minute', '30 minute interval', 'Vehicle count')
avg_on_day_of_week(FLOW_FEATURE, 'Average traffic flow on each day of week every 30 minute', '30 minute interval', 'traffic flow')




NameError: name 'df' is not defined

## Average number of each type of vehicle in each hour of day

In [12]:
def avg_on_hour(selected_attr, title='title', xla='x', yla='y'):
    byhour = df.groupby(df['half_hour_interval'])[selected_attr].mean()
    fig = px.line(byhour, y=selected_attr, x=byhour.index)
    fig.update_xaxes(
        tickmode='linear'
    )
    fig.update_layout(
        title=title,
        xaxis_title=xla,
        yaxis_title=yla
    )
    fig.show()

avg_on_hour(VEHICLE_FEATURE, 'Average vehicle type count every 30 minute of day', '30 minute interval', 'Vehicle count')
avg_on_hour(FLOW_FEATURE, 'Average traffic flow 30 minute of day', '30 minute interval', 'traffic flow')

## Average number of each vehicle type in each weather condition. 

In [13]:
def avg_on_weather_conditiopn(selected_attr, title='title', xla='x', yla='y'):
    notime = df.drop(columns=['time'])
    weather = notime.groupby('weatherDesc')[selected_attr].mean()

    fig = px.bar(weather, y=selected_attr, x=weather.index, barmode='group')
    fig.update_layout(
        title=title,
        xaxis_title=xla,
        yaxis_title=yla
    )
    fig.show()

avg_on_weather_conditiopn(VEHICLE_FEATURE, 'Average vehicle type count on each weather condition', 'weather condition', 'Vehicle count')
avg_on_weather_conditiopn(FLOW_FEATURE, 'Average traffic flow on each weather condition', 'weather condition', 'traffic flow')

## Convert hour to float

In [14]:
train_df = df.copy()

train_df['hour'] = (((train_df['time'].dt.hour * 60) + train_df['time'].dt.minute) // 30) / 2

train_df['weatherDesc'].unique()

array(['scattered clouds', 'few clouds', 'overcast clouds', 'light rain',
       'moderate rain', 'broken clouds', 'clear sky',
       'heavy intensity rain', 'haze', 'mist'], dtype=object)

## Convert day to is_weekend

In [15]:
def categorize_weather(wether_desc):
    
    if wether_desc in ['clear sky', 'few clouds', 'scattered clouds']:
        return 0
    
    if wether_desc in ['broken clouds', 'overcast clouds']:
        return 1
    
    if wether_desc in ['haze', 'mist']:
        return 2
    
    if wether_desc in ['light rain', 'moderate rain', 'heavy intensity rain']:
        return 3

train_df['ec_weather'] = train_df['weatherDesc'].map(categorize_weather)

train_df['is_weekend'] = train_df['Day'].map(
    lambda day: day in ['Saturday', 'Sunday']
).astype('int8')
train_df

time  currentSpeed       weatherDesc  motorcycles  cars  \
0   2025-03-12 10:30:03            46  scattered clouds            0    13   
1   2025-03-12 11:00:04            49  scattered clouds            1     9   
2   2025-03-12 11:30:04            45  scattered clouds            1    13   
3   2025-03-12 12:00:04            49  scattered clouds            0     5   
4   2025-03-12 12:30:04            51  scattered clouds            4    11   
..                  ...           ...               ...          ...   ...   
928 2025-04-01 22:01:40            66        few clouds            0     0   
929 2025-04-01 22:31:40            66        few clouds            0     1   
930 2025-04-01 23:01:40            66        few clouds            0     0   
931 2025-04-01 23:31:40            66        light rain            0     3   
932 2025-04-02 00:01:40            66        few clouds            0     0   

     heavyVehicles half_hour_interval  total_vehicle        Day  hour  \
0                0              10:30             13  Wednesday  10.5   
1                0              11:00             10  Wednesday  11.0   
2                0              11:30             14  Wednesday  11.5   
3                0              12:00              5  Wednesday  12.0   
4                0              12:30             15  Wednesday  12.5   
..             ...                ...            ...        ...   ...   
928              0              22:00              0    Tuesday  22.0   
929              0              22:30              1    Tuesday  22.5   
930              0              23:00              0    Tuesday  23.0   
931              0              23:30              3    Tuesday  23.5   
932              0              00:00              0  Wednesday   0.0   

     ec_weather  is_weekend  
0             0           0  
1             0           0  
2             0           0  
3             0           0  
4             0           0  
..          ...         ...  
928           0           0  
929           0           0  
930           0           0  
931           3           0  
932           0           0  

[933 rows x 12 columns]

## Creating sequence function

In [16]:
def create_seq(data, seq_length):
    seq = []
    target = []
    for i in range(len(data) - seq_length):
        seq.append(data[i: i + seq_length, :-1])
        target.append(data[i: i + seq_length, -1])
    return torch.FloatTensor(seq), torch.FloatTensor(target)

## Create model class

In [17]:
class TrafficGRU(nn.Module):
    def __init__(self, weather_cat, hidden_size=64, num_layers=2):
        super().__init__()
        
        input_size = len(weather_cat) + 2 # Number of weather condition + sin/cos hour + is_weekend
        self.weather_cat_count = len(weather_cat)
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1) 
        self.ln = nn.LayerNorm(hidden_size) 
        self.relu = nn.ReLU(inplace=True)   

    def preprocess_param(self, hour, weather):

        sin_hour = torch.sin(2 * np.pi * hour / 24).unsqueeze(0)
        sin_cos = torch.cos(2 * np.pi * hour / 24).unsqueeze(0)
        
        int_weather = int(weather.item())
        weather_one_hot = torch.nn.functional.one_hot(
            torch.tensor(int_weather),
            num_classes=self.weather_cat_count
        ).float()
        
        return torch.cat([sin_cos, sin_hour, weather_one_hot])
        
    def forward(self, x):
        
        processed = []
        for i in x:
            
            batch = []
            
            for j in i:
                
                hour, weather = j[0], j[1]
                feature = self.preprocess_param(hour, weather)
                batch.append(feature)
            
            processed.append(torch.stack(batch))
        
        gru_input = torch.stack(processed)
        
        h0 = torch.zeros(self.gru.num_layers, x.size(0), self.gru.hidden_size)
        out, _ = self.gru(gru_input , h0)
        out = self.ln(out)
        out = self.fc(out[:, -1, :])
        out = self.relu(out)
        return out

# Create and training model function

In [18]:
def create_and_train_model(target_feature, epoch, lr=0.001):
    weather_cat = list(train_df['ec_weather'].unique())

    # Set up hyper parameter
    hidden_size = 64
    num_layers = 2

    train_df.sort_values('time', inplace=True)
    train_set, car_test = np.split(train_df[['hour', 'ec_weather'] + [target_feature]], [int(len(train_df) * 0.8)])
    X, y = create_seq(train_set.values, 2)

    model = TrafficGRU(weather_cat, hidden_size, num_layers)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    dataset = TensorDataset(X, y)
    dataloader = DataLoader(dataset, batch_size=24)

    for e in range(epoch): 
        for batch_x, batch_y in dataloader:
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print(f'Epoch {e+1}, Loss: {loss.item():.4f}')    

    X_test, y_test = create_seq(car_test.values, 2)
    test_criterion = nn.MSELoss()

    with torch.no_grad():
        predictions = model(X_test)
        test_loss = test_criterion(predictions, y_test)
    
    return  model, test_loss.item()    

# Train model of each target feature

In [31]:
car_model, car_per = create_and_train_model('cars', 200)
print("Number of car prediction model loss: ", car_per)

/Users/Ohm/Desktop/ske/daq/traffic-flow-analysis/.venv/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning:

'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.

/Users/Ohm/Desktop/ske/daq/traffic-flow-analysis/.venv/lib/python3.13/site-packages/torch/nn/modules/loss.py:610: UserWarning:

Using a target size (torch.Size([24, 2])) that is different to the input size (torch.Size([24, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.



Epoch 1, Loss: 383.9402
Epoch 2, Loss: 348.2321
Epoch 3, Loss: 320.4049
Epoch 4, Loss: 298.6758
Epoch 5, Loss: 283.6964
Epoch 6, Loss: 273.2932
Epoch 7, Loss: 268.1712
Epoch 8, Loss: 266.2321
Epoch 9, Loss: 263.3159
Epoch 10, Loss: 256.8588
Epoch 11, Loss: 251.3123
Epoch 12, Loss: 247.8071
Epoch 13, Loss: 244.9009
Epoch 14, Loss: 241.6092
Epoch 15, Loss: 239.2289
Epoch 16, Loss: 237.2160
Epoch 17, Loss: 235.4085
Epoch 18, Loss: 233.8389
Epoch 19, Loss: 232.4606
Epoch 20, Loss: 231.2470
Epoch 21, Loss: 230.1565
Epoch 22, Loss: 229.1683
Epoch 23, Loss: 228.2423
Epoch 24, Loss: 227.3685
Epoch 25, Loss: 226.5402
Epoch 26, Loss: 225.7579
Epoch 27, Loss: 225.0205
Epoch 28, Loss: 224.3486
Epoch 29, Loss: 223.7010
Epoch 30, Loss: 223.0965
Epoch 31, Loss: 222.5366
Epoch 32, Loss: 222.0125
Epoch 33, Loss: 221.5473
Epoch 34, Loss: 221.0947
Epoch 35, Loss: 220.6790
Epoch 36, Loss: 220.2897
Epoch 37, Loss: 219.9312
Epoch 38, Loss: 219.5940
Epoch 39, Loss: 219.2814
Epoch 40, Loss: 218.9884
Epoch 41,

/Users/Ohm/Desktop/ske/daq/traffic-flow-analysis/.venv/lib/python3.13/site-packages/torch/nn/modules/loss.py:610: UserWarning:

Using a target size (torch.Size([185, 2])) that is different to the input size (torch.Size([185, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.



In [32]:
mot_model, mot_per = create_and_train_model('motorcycles', 50)
print("Number of motorcycles prediction model loss: ", mot_per)

/Users/Ohm/Desktop/ske/daq/traffic-flow-analysis/.venv/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning:

'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.

/Users/Ohm/Desktop/ske/daq/traffic-flow-analysis/.venv/lib/python3.13/site-packages/torch/nn/modules/loss.py:610: UserWarning:

Using a target size (torch.Size([24, 2])) that is different to the input size (torch.Size([24, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.



Epoch 1, Loss: 4.8359
Epoch 2, Loss: 4.5384
Epoch 3, Loss: 4.5247
Epoch 4, Loss: 4.4913
Epoch 5, Loss: 4.4261
Epoch 6, Loss: 4.3094
Epoch 7, Loss: 4.0398
Epoch 8, Loss: 3.6043
Epoch 9, Loss: 3.2897
Epoch 10, Loss: 3.1085
Epoch 11, Loss: 2.9902
Epoch 12, Loss: 2.8981
Epoch 13, Loss: 2.8383
Epoch 14, Loss: 2.7933
Epoch 15, Loss: 2.7688
Epoch 16, Loss: 2.7444
Epoch 17, Loss: 2.7199
Epoch 18, Loss: 2.7203
Epoch 19, Loss: 2.7565
Epoch 20, Loss: 2.7893
Epoch 21, Loss: 2.8025
Epoch 22, Loss: 2.8036
Epoch 23, Loss: 2.7989
Epoch 24, Loss: 2.7972
Epoch 25, Loss: 2.7927
Epoch 26, Loss: 2.7959
Epoch 27, Loss: 2.7898
Epoch 28, Loss: 2.7981
Epoch 29, Loss: 2.7889
Epoch 30, Loss: 2.7989
Epoch 31, Loss: 2.7906
Epoch 32, Loss: 2.7976
Epoch 33, Loss: 2.7927
Epoch 34, Loss: 2.7965
Epoch 35, Loss: 2.7937
Epoch 36, Loss: 2.7959
Epoch 37, Loss: 2.7939
Epoch 38, Loss: 2.7952
Epoch 39, Loss: 2.7936
Epoch 40, Loss: 2.7945
Epoch 41, Loss: 2.7931
Epoch 42, Loss: 2.7938
Epoch 43, Loss: 2.7927
Epoch 44, Loss: 2.79

/Users/Ohm/Desktop/ske/daq/traffic-flow-analysis/.venv/lib/python3.13/site-packages/torch/nn/modules/loss.py:610: UserWarning:

Using a target size (torch.Size([185, 2])) that is different to the input size (torch.Size([185, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.



In [35]:
heavy_model, heavy_per = create_and_train_model('heavyVehicles', 50)
print("Number of heavy vehicles prediction model loss: ", heavy_per)

/Users/Ohm/Desktop/ske/daq/traffic-flow-analysis/.venv/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning:

'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.

/Users/Ohm/Desktop/ske/daq/traffic-flow-analysis/.venv/lib/python3.13/site-packages/torch/nn/modules/loss.py:610: UserWarning:

Using a target size (torch.Size([24, 2])) that is different to the input size (torch.Size([24, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.



Epoch 1, Loss: 0.5791
Epoch 2, Loss: 0.5374
Epoch 3, Loss: 0.5749
Epoch 4, Loss: 0.5714
Epoch 5, Loss: 0.5669
Epoch 6, Loss: 0.5684
Epoch 7, Loss: 0.5684
Epoch 8, Loss: 0.5699
Epoch 9, Loss: 0.5723
Epoch 10, Loss: 0.5755
Epoch 11, Loss: 0.5785
Epoch 12, Loss: 0.5811
Epoch 13, Loss: 0.5825
Epoch 14, Loss: 0.5829
Epoch 15, Loss: 0.5830
Epoch 16, Loss: 0.5826
Epoch 17, Loss: 0.5808
Epoch 18, Loss: 0.5793
Epoch 19, Loss: 0.5780
Epoch 20, Loss: 0.5769
Epoch 21, Loss: 0.5759
Epoch 22, Loss: 0.5750
Epoch 23, Loss: 0.5727
Epoch 24, Loss: 0.5716
Epoch 25, Loss: 0.5702
Epoch 26, Loss: 0.5688
Epoch 27, Loss: 0.5674
Epoch 28, Loss: 0.5662
Epoch 29, Loss: 0.5652
Epoch 30, Loss: 0.5644
Epoch 31, Loss: 0.5639
Epoch 32, Loss: 0.5634
Epoch 33, Loss: 0.5630
Epoch 34, Loss: 0.5627
Epoch 35, Loss: 0.5625
Epoch 36, Loss: 0.5624
Epoch 37, Loss: 0.5623
Epoch 38, Loss: 0.5623
Epoch 39, Loss: 0.5622
Epoch 40, Loss: 0.5621
Epoch 41, Loss: 0.5620
Epoch 42, Loss: 0.5619
Epoch 43, Loss: 0.5619
Epoch 44, Loss: 0.56

/Users/Ohm/Desktop/ske/daq/traffic-flow-analysis/.venv/lib/python3.13/site-packages/torch/nn/modules/loss.py:610: UserWarning:

Using a target size (torch.Size([185, 2])) that is different to the input size (torch.Size([185, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.



In [37]:
speed_model, speed_per = create_and_train_model('currentSpeed', 200, 0.01)
print("Traffic flow prediction model loss: ", speed_per)

/Users/Ohm/Desktop/ske/daq/traffic-flow-analysis/.venv/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning:

'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.

/Users/Ohm/Desktop/ske/daq/traffic-flow-analysis/.venv/lib/python3.13/site-packages/torch/nn/modules/loss.py:610: UserWarning:

Using a target size (torch.Size([24, 2])) that is different to the input size (torch.Size([24, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.



Epoch 1, Loss: 385.8145
Epoch 2, Loss: 734.3572
Epoch 3, Loss: 378.5198
Epoch 4, Loss: 507.5997
Epoch 5, Loss: 505.2856
Epoch 6, Loss: 508.5864
Epoch 7, Loss: 507.4730
Epoch 8, Loss: 509.9268
Epoch 9, Loss: 498.8421
Epoch 10, Loss: 494.5690
Epoch 11, Loss: 487.7369
Epoch 12, Loss: 485.2652
Epoch 13, Loss: 483.5807
Epoch 14, Loss: 481.4706
Epoch 15, Loss: 480.0526
Epoch 16, Loss: 477.6383
Epoch 17, Loss: 476.2703
Epoch 18, Loss: 474.1281
Epoch 19, Loss: 468.8483
Epoch 20, Loss: 471.5514
Epoch 21, Loss: 469.0316
Epoch 22, Loss: 462.9005
Epoch 23, Loss: 472.1873
Epoch 24, Loss: 464.4713
Epoch 25, Loss: 464.4107
Epoch 26, Loss: 466.4472
Epoch 27, Loss: 461.2355
Epoch 28, Loss: 465.4388
Epoch 29, Loss: 459.7578
Epoch 30, Loss: 459.9160
Epoch 31, Loss: 458.2886
Epoch 32, Loss: 446.0283
Epoch 33, Loss: 450.4316
Epoch 34, Loss: 452.9865
Epoch 35, Loss: 458.9875
Epoch 36, Loss: 449.1101
Epoch 37, Loss: 454.7064
Epoch 38, Loss: 440.1561
Epoch 39, Loss: 446.6269
Epoch 40, Loss: 448.7911
Epoch 41,

/Users/Ohm/Desktop/ske/daq/traffic-flow-analysis/.venv/lib/python3.13/site-packages/torch/nn/modules/loss.py:610: UserWarning:

Using a target size (torch.Size([185, 2])) that is different to the input size (torch.Size([185, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.



In [38]:
total_model, total_per = create_and_train_model('total_vehicle', 100)
print("Total number of vehicles prediction model loss: ", total_per)

/Users/Ohm/Desktop/ske/daq/traffic-flow-analysis/.venv/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning:

'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.

/Users/Ohm/Desktop/ske/daq/traffic-flow-analysis/.venv/lib/python3.13/site-packages/torch/nn/modules/loss.py:610: UserWarning:

Using a target size (torch.Size([24, 2])) that is different to the input size (torch.Size([24, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.



Epoch 1, Loss: 487.6703
Epoch 2, Loss: 442.1810
Epoch 3, Loss: 401.5064
Epoch 4, Loss: 370.4901
Epoch 5, Loss: 346.6370
Epoch 6, Loss: 327.5991
Epoch 7, Loss: 315.9033
Epoch 8, Loss: 308.8261
Epoch 9, Loss: 307.0090
Epoch 10, Loss: 303.9880
Epoch 11, Loss: 299.8248
Epoch 12, Loss: 293.1603
Epoch 13, Loss: 289.0976
Epoch 14, Loss: 285.2636
Epoch 15, Loss: 280.8961
Epoch 16, Loss: 277.4226
Epoch 17, Loss: 274.2808
Epoch 18, Loss: 271.4032
Epoch 19, Loss: 268.8130
Epoch 20, Loss: 266.4641
Epoch 21, Loss: 264.3406
Epoch 22, Loss: 262.4068
Epoch 23, Loss: 260.6497
Epoch 24, Loss: 259.0443
Epoch 25, Loss: 257.5802
Epoch 26, Loss: 256.2346
Epoch 27, Loss: 255.0026
Epoch 28, Loss: 253.8642
Epoch 29, Loss: 252.8374
Epoch 30, Loss: 251.9515
Epoch 31, Loss: 251.0820
Epoch 32, Loss: 250.6910
Epoch 33, Loss: 250.1015
Epoch 34, Loss: 249.4745
Epoch 35, Loss: 248.8856
Epoch 36, Loss: 248.3180
Epoch 37, Loss: 247.7990
Epoch 38, Loss: 247.3072
Epoch 39, Loss: 246.8633
Epoch 40, Loss: 246.4434
Epoch 41,

/Users/Ohm/Desktop/ske/daq/traffic-flow-analysis/.venv/lib/python3.13/site-packages/torch/nn/modules/loss.py:610: UserWarning:

Using a target size (torch.Size([185, 2])) that is different to the input size (torch.Size([185, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.



# Prediction model performance (MSE)

In [63]:
from math import sqrt

print(f"""
MSE of:
Number of car prediction model: {car_per:.4}
Number of motorcycle prediction model: {mot_per:.4}
Number of heavy vehicle prediction model: {heavy_per:.4}
Total number vehicle prediction model: {total_per:.4}
Traffic flow prediction model: {speed_per:.4}
""")


MSE of:
Number of car prediction model: 15.66
Number of motorcycle prediction model: 2.357
Number of heavy vehicle prediction model: 0.345
Total number vehicle prediction model: 19.84
Traffic flow prediction model: 38.15



In [64]:
import os

def export_model(model, model_name): 
    model.eval()
    scripted_model = torch.jit.script(model)  # or torch.jit.trace(model, input_example)
    
    if (not os.path.isdir(MODEL_DIR)):
        os.mkdir(MODEL_DIR)
    scripted_model.save(f"{MODEL_DIR}/{model_name}.pt")

if (CREATE_MODEL):
    export_model(car_model, "car_num_model")
    export_model(mot_model, "motorcycle_vehicle_num_model")
    export_model(heavy_model, "heavy_vehicle_num_model")
    export_model(total_model, "total_vehicle_num_model")
    export_model(speed_model, "traffic_flow_model")